<a href="https://colab.research.google.com/github/tmu-nlp/100knock2021/blob/main/wei/chapter08/knock71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task Description   
71. 単層ニューラルネットワークによる予測

問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．
$$\hat{y}_1=softmax(x_1W),\\\hat{Y}=softmax(X_{[1:4]}W)$$
$X_{[1:4]}∈\mathbb{R}^{4×d}$は特徴ベクトル$x_1$,$x_2$,$x_3$,$x_4$を縦に並べた行列である．
* 行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化
* $\hat{\boldsymbol y_1} \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトル
* 学習事例集合の場合では、各カテゴリに属する確率を行列$\hat{Y} \in \mathbb{R}^{n \times L}$として表現


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('drive/MyDrive/ColabNotebooks/NLPknock100/newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

In [5]:
from gensim.models import KeyedVectors
import torch
import string

# 学習済み単語ベクトルを読み込む
model = KeyedVectors.load_word2vec_format('drive/MyDrive/ColabNotebooks/NLPknock100/GoogleNews-vectors-negative300.bin.gz', binary=True)

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
  vec = [model[word] for word in words if word in model]  # 1語ずつベクトル化

  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])

In [6]:
# SGLNetという単層ニューラルネットワークを定義
from torch import nn

class SGLNet(nn.Module):
  #　ネットのlayerを定義
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size, bias=False)
    nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化
  #　forwardで入力データが順伝播時に通るレイヤーを順に配置しておく
  def forward(self, x):
    x = self.fc(x)
    return x


In [7]:
# 単層ニューラルネットワークの初期化
SigelNNmodel = SGLNet(300, 4) 
#未学習の行列Wで事例x_1を分類したとき，各カテゴリに属する確率を表すベクトル
y_hat_1 = torch.softmax(SigelNNmodel(X_train[:1]), dim=-1)
print(y_hat_1)

tensor([[0.2993, 0.0426, 0.4832, 0.1749]], grad_fn=<SoftmaxBackward>)


In [8]:
Y_hat = torch.softmax(SigelNNmodel.forward(X_train[:4]), dim=-1)
print(Y_hat)

tensor([[0.2993, 0.0426, 0.4832, 0.1749],
        [0.2223, 0.6417, 0.1280, 0.0080],
        [0.1510, 0.6303, 0.1328, 0.0859],
        [0.1233, 0.3885, 0.1048, 0.3834]], grad_fn=<SoftmaxBackward>)
